In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import os
import sys
import time
import logging
import re
from commons import *
from gan_arch import *
from datetime import datetime

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
    print("No compatible GPUs found")


In [ ]:
DATETIME =  datetime.now().strftime("%Y%m%d-%H%M%S")
METU_RAW_PATH = '/qarr/studia/magister/datasets/METU/930k_logo_v3/'
METU_DATASET_PATH = '/home/zenfur/magister/resized_930k_logo/'
EVAL_ORIGIN_PATH = '/qarr/studia/magister/datasets/METU/query_reversed/'
EVAL_DATASET_PATH = '/home/zenfur/magister/metu_eval_256sq/'
LOG_DIR = "siamese_logs/" + DATETIME
MODEL_SAVE_NAME = "siamese_model" + DATETIME
LOAD = True
LAST_MODEL = "/home/zenfur/magister/jupyter/siamese_model20210322-032225_1560/"
TESTING=False
    
last_epoch=0
#tf.debugging.experimental.enable_dump_debug_info("siamese_logs/", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

# Preparing the dataset pipeline and testing

In [ ]:
imagesList = tf.io.matching_files(EVAL_DATASET_PATH + "*.jpg")

@tf.function
def tf_get_filename(path):
    return tf.strings.regex_replace(path, "[^/]*/", "")


#@tf.function
def tf_read_image(path):
    # Retrieving the group number from file name
    img = tf.io.read_file(path)
    return tf.image.decode_jpeg(img, channels=3, dct_method='INTEGER_ACCURATE')


def tf_get_class_from_name(path):
    filename = tf_get_filename(path)
    group_number = tf.strings.to_number(
        tf.strings.regex_replace(filename, "-.*$", ""), 
        out_type=tf.dtypes.int32
    )
    return group_number

#@tf.function
def tf_convert_and_normalize_img(img):
    c = tf.constant(256.0, dtype=tf.dtypes.float32)
    img = tf.cast(img, tf.dtypes.float32)
    return tf.math.divide(img, c)


evalpathsDB = tf.data.Dataset.from_tensor_slices(imagesList)

DBlen = len(imagesList)

evalDB = (      evalpathsDB.map(tf_read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 .batch(32)
                 .map(tf_convert_and_normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)

evalGrps = (      evalpathsDB.map(tf_get_class_from_name, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                   .batch(32)
)


# Counting and preparing images into groups by name
By convention, discarding images from group 0, as they have been manually inserted as the examples that differ from the rest sampled from 930k METU dataset.

Reordering the labels from alphabetical order into ascending by group number order.

In [ ]:
labs = np.array(list(evalGrps.unbatch().as_numpy_iterator()))
labsOrder = np.argsort(labs)
labs = labs[labsOrder]

In [ ]:
imageGroupsDct = dict()
for i, l in enumerate(labs):
    last_count = imageGroupsDct.get(l,(i,0))
    imageGroupsDct[l] = (last_count[0], last_count[1] + 1)
del(imageGroupsDct[0])


### Custom triples sampler based on linear congruent generator 
The generator has period equal NumberOfUnlikeSamples*(NumberOfAlikeSamples-1)

In [ ]:
def triplet_generator(start, length, avoid, n, initial, prime=756212269):
    """
    results in sequence of triplets (<start:start+length>, <avoid>, <0:n-1 excluding start:start+length-1>)
    """
    start = np.array(start, dtype=np.int32)
    length = np.array(length, dtype=np.int32)
    avoid = np.array(avoid, dtype=np.int32)
    initial = np.array(initial, dtype=np.int32)
    current = initial
    unlikeCount = (n-length)
    length = length - 1
    modulo_base = length * unlikeCount
    multiplier = modulo_base*11*3+1
    while True:
        # Generating next modulo from sequence
        current = (multiplier * current + prime) % modulo_base
        like = (current % length)
        unlike = current // length
        
        # Calculating proper indices from random modulos
        like += start
        like += (like >= avoid)
        like = np.expand_dims(like, axis=1)
        
        unlike = unlike + ((unlike >= start) * (length+1))
        unlike = np.expand_dims(unlike, axis=1)
        
        for triplet in np.concatenate((like, np.expand_dims(avoid, axis=1), unlike), axis=1):
            yield triplet

Testing the generator 

In [ ]:
if TESTING:
    gen = triplet_generator(starts, lengths, avoids, len(labs), seeds)
    # Test: group 1:3, n=6, x = 2
    g = triplet_generator([1], [3], [2], 6, [423432231])
    s = set()
    for i in range(2*3):
        t = next(g)
        print(t)
        if tuple(t) in s:
            break
        else:
            s.add(tuple(t))
    assert i == (2*3)-1

In [ ]:
if TESTING:
    for i in range(10):
        print(next(gen))

### Splitting the dataset into validation and training subsets

Taking **validationUniques** samples from each class as uniqiue anchor samples in validation dataset. Pairing those with **xSamples** random samples from original class as alike sample and any other class as unlike sample.

In [ ]:
validationUniques = 2
validationSubset = [(a, [i for i in range(j[0],j[0] + validationUniques)]) for a,j in imageGroupsDct.items()]
validationSamples = []
xSamples = 40
N = len(labs)
np.random.seed(14200)
for grp, samples in validationSubset:
    for sample in samples:
        groupStart =  imageGroupsDct[grp][0]
        groupLength = imageGroupsDct[grp][1]
        for i in range(xSamples):
            alike = np.random.randint(groupStart, groupStart + groupLength)
            unlike = np.random.randint(0, N-groupLength)
            unlike += (unlike>=groupStart)*groupLength
            validationSamples.append((alike, sample, unlike))

In [ ]:
trainingTranslationTable = np.array(range(N-validationUniques*len(validationSubset)))
starts = [s[0] for s in imageGroupsDct.values()]
starts.sort()
starts.append(N+1)
j = 0
for i in range(len(trainingTranslationTable)):
    if starts[j] <= i+j*validationUniques:
        j += 1
    trainingTranslationTable[i] = i+j*validationUniques
    

In [ ]:
imageGroups = list(imageGroupsDct.items())
avoids, starts, lengths, seeds = [], [], [], []

# Generating the random seeds for custom random sequence generators that iterate over triplets from each group
# For the sake of being repeatable, fixing seed
np.random.seed(949127843)
for igrp in imageGroups:
    for i in range(igrp[1][1] - validationUniques):
        avoids.append(i+igrp[1][0] - (igrp[0]-1)*validationUniques)
        starts.append(igrp[1][0] - (igrp[0]-1)*validationUniques)
        lengths.append(igrp[1][1] - validationUniques)
        seeds.append(np.random.randint(0, high=10000000))

In [ ]:
trainLength = len(trainingTranslationTable)
validSamples = N - trainLength
validLength = len(validationSamples)

In [ ]:
validLength

## Building the pipeline segments 

Converting the database of samples into numpy array, since it can fit into RAM memory to save time.

In [ ]:
imagesTable = tf_db_to_array(evalDB, DBlen)
imagesTable = imagesTable[labsOrder]

In [ ]:
imagesTable = tf.constant(imagesTable)

In [ ]:
trainingTranslationTable = tf.constant(trainingTranslationTable)

In [ ]:
def get_triplet_by_index(triplet):
    # @triplet: tuple/tensor of indices in the images table
    return tf.gather(imagesTable, triplet)


def translate_indices(triplet):
    return tf.gather(trainingTranslationTable, triplet)

Initialising the common source of random numbers

In [ ]:
rng = tf.random.Generator.from_seed(41431)
rngValid = tf.random.Generator.from_seed(198489)

Defining the data augmentation functions for the pipeline.

In [ ]:
def random_slice_224x224(image, seed):
    return tf.image.stateless_random_crop(image, [224,224,3], seed)


def get_center_slice(image):
    return tf.image.central_crop(image, 224/256)


def random_rotate(image, seed):
    if seed > 3:
        seed = 0
    return tf.image.rot90(image, k=seed)


def static_augment(image, seeds):
    sditer = iter(seeds)
    
    image = random_slice_224x224(image, next(sditer))
    image = random_rotate(image, next(sditer))
    
    return image
    
    
def augment(image):
    seeds = [rng.make_seeds(2)[0], rng.uniform([], minval=0, maxval=5, dtype=tf.int32)]
    return static_augment(image, seeds)


### The input data pipeline

Defining the training dataset

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 4
trainDset = tf.data.Dataset.from_generator(triplet_generator,
                                    args = [starts, lengths, avoids, trainLength, seeds],
                                    output_signature=tf.TensorSpec(shape=(3), dtype=tf.int32))

trainDset = (trainDset.shuffle(trainLength)
      .map(translate_indices, num_parallel_calls=AUTOTUNE, deterministic=True)
      .map(get_triplet_by_index)
      .unbatch()
# unbatching to augment images individually, as its difficult to make a parallel function for it
# TODO: possible improvement
      .map(augment, num_parallel_calls=AUTOTUNE, deterministic=True)
      .batch(3*BATCH_SIZE)
      .prefetch(2)#AUTOTUNE)
)

Defining the validation dataset

In [ ]:
# Freezing augmentation seeds for validation d-set or disabling it alltogether

# validRandoms = []
# for s in range(validLength):
#     validRandoms.append([rng.make_seeds(2)[0], rng.uniform([], minval=0, maxval=5, dtype=tf.int32)])

validDset = (tf.data.Dataset.from_tensor_slices(validationSamples)
                .repeat()
                .map(get_triplet_by_index)
                .unbatch()
                .map(get_center_slice) # disabling augmentation
                .batch(3*BATCH_SIZE)
                .prefetch(2)
            )

In [ ]:
t = validDset.take(16)

In [ ]:
g = next(iter(t))

In [ ]:
plt.imshow(g[11])

Sampling and testing the pipeline

In [ ]:
if TESTING:
    img = get_triplet_by_index(tf.constant(next(triplet_generator(starts, lengths, avoids, len(labs), seeds))))
    f, subplots = plt.subplots(1,3)
    for i, sb in enumerate(subplots):
        sb.imshow(img[i])
        


In [ ]:
# if TESTING:
#     img = df.take(1)
#     img = next(iter(img))
#     f, subplots = plt.subplots(1,3)
#     for i, sb in enumerate(subplots):
#         sb.imshow(img[i])

## Siamese model

Importing the  pre-trained VGG16 model with weights from imagenet without classification part.

Adding 2 dense layers on top of convolutions for 4096 representation.

In [ ]:
if LOAD:
    siamese_base = tf.keras.models.load_model(LAST_MODEL)
else:
    vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
#     input_shape=None,
#     pooling=None,
    )
    #vgg16.trainable = False
    siamese_base = tf.keras.models.Sequential()
    for layer in (vgg16,
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(4096, activation='relu'),
                    tf.keras.layers.Dense(4096, activation='relu')
                 ):
        siamese_base.add(layer)

In [ ]:
#tf.keras.utils.plot_model(vgg16, "vgg16-base.png", show_shapes=True)

In [ ]:
#tf.keras.utils.plot_model(siamese_base, "siamese.png", show_shapes=True)

### Siamese triplet loss function 

In [ ]:
def triplet_loss(alike, anchor, unlike, margin=1.0, reduce=tf.reduce_mean):
    a = tf.norm(alike-anchor, axis=1)
    b = tf.norm(unlike-anchor, axis=1)
    return reduce(tf.maximum(a + margin - b, 0.0))

## Defining custom model

In [ ]:
def describe(x):
    try:
        return f'{x.shape}'
    except AttributeError:
        return f"{'[' + ', '.join([describe(q) for q in x]) + ']'}"
    
class TripletSiamese(tf.keras.models.Model):
    def __init__(self, shared_net, name=None):
        super(TripletSiamese, self).__init__(name=name)
        self.siamese_base = shared_net
        self.callctr = 0

    def compile(self, optimizer, loss_margin):
        super(TripletSiamese, self).compile()
        self.optimizer = optimizer
        self.loss = triplet_loss#lambda a,b,c: triplet_loss(a,b,c, margin=loss_margin)
    
    def normalize_output(self, x):
        return x / tf.expand_dims(tf.maximum(tf.math.reduce_max(x, axis=1), 1e-5), axis=1)
    
    @tf.function#(jit_compile=True)
    def train_step(self, input_triplets):
        
        with tf.GradientTape(persistent=True) as tape:
            # training and calculating the error function gradient
            representations = self.siamese_base(input_triplets, training=True)
            representations = self.normalize_output(representations)
            alike = tf.strided_slice(representations, [0,0], tf.shape(representations), strides=[3,1])
            anchor = tf.strided_slice(representations, [1,0], tf.shape(representations), strides=[3,1])
            unlike = tf.strided_slice(representations, [2,0], tf.shape(representations), strides=[3,1])
            loss = self.loss(alike, anchor, unlike)
        grads = tape.gradient(loss, self.siamese_base.trainable_weights)

        
#         alike_in = tf.strided_slice(input_triplets, [0,0,0,0], tf.shape(input_triplets), strides=[3,1,1,1])
#         anchor_in = tf.strided_slice(input_triplets, [1,0,0,0], tf.shape(input_triplets), strides=[3,1,1,1])
#         unlike_in = tf.strided_slice(input_triplets, [2,0,0,0], tf.shape(input_triplets), strides=[3,1,1,1])

#         with tf.GradientTape() as tape:
#             alike = self.siamese_base(alike_in, training=True)
#         grads1 = tape.gradient(alike, self.siamese_base.trainable_weights) 
        
#         with tf.GradientTape() as tape:
#             anchor = self.siamese_base(anchor_in, training=True)
#         grads2 = tape.gradient(anchor, self.siamese_base.trainable_weights) 
        
#         with tf.GradientTape() as tape:
#             unlike = self.siamese_base(unlike_in, training=True)
#         grads3 = tape.gradient(unlike, self.siamese_base.trainable_weights) 
        
#         tf.print(describe(grads1))
#         tf.print(describe(alike))
#         tf.print(describe(anchor))
#         grads1 *= 2*(alike - anchor)
#         grads2 *= 2*(unlike - alike)
#         grads3 *= 2*(alike - unlike)
        
#         grads = np.mean([grads1, grads2, grads3])# ...

#         loss = self.loss(alike, anchor, unlike)
            
        self.optimizer.apply_gradients(zip(grads, self.siamese_base.trainable_weights))
        
        return {"loss": loss}
    
        
    def evaluate(self, x=None, y=None, batch_size=None, verbose=False, sample_weight=None, steps=None,
                callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False,
                return_dict=False):
        r = self.siamese_base.predict(x=x, batch_size=batch_size, verbose=False, steps=steps, callbacks=callbacks)
        r = self.normalize_output(r)
        alike = tf.strided_slice(r, [0,0], tf.shape(r), strides=[3,1])
        anchor = tf.strided_slice(r, [1,0], tf.shape(r), strides=[3,1])
        unlike = tf.strided_slice(r, [2,0], tf.shape(r), strides=[3,1])
        dct = {"loss":self.loss(alike, anchor, unlike, reduce=tf.reduce_mean)}
        if callbacks is not None:
            for cb in callbacks:
                cb.on_test_batch_end(x, logs=dct)
        if return_dict:
            return dct
        else:
            return dct["loss"]


#     def __call__(self, x, training=False):
#         representations = self.siamese_base(x, training=training)
#         representations = self.normalize_output(representations)
#         alike = tf.strided_slice(representations, [0,0], tf.shape(representations), strides=[3,1])
#         anchor = tf.strided_slice(representations, [1,0], tf.shape(representations), strides=[3,1])
#         unlike = tf.strided_slice(representations, [2,0], tf.shape(representations), strides=[3,1])
#         return self.loss(alike, anchor, unlike)
    
    def save(self, path):
        self.siamese_base.save(path)

## Initialising and compiling the model

In [ ]:
triplet_model = TripletSiamese(siamese_base, name=MODEL_SAVE_NAME)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
triplet_model.compile(opt, triplet_loss)

In [ ]:
# triplet_model.train_step(tf.image.central_crop(img, 224/256))
# #siamese_base(tf.image.central_crop(img, 224/256))

# Calling the training with hooks

### Preparing custom callbacks

In [ ]:
class MeanTBCallback(tf.keras.callbacks.TensorBoard):
    def __init__(self, *args, **kwargs):
        super(MeanTBCallback, self).__init__(*args, **kwargs)
        self._epoch = 1
        self.mean_train_loss = 0
        self.mean_test_loss = 0
        self.train_batches = 0
        self.test_batches = 0

    def on_epoch_end(self, epoch, logs=None):
        super(MeanTBCallback, self).on_epoch_end(epoch, logs=logs)
        # Tensorflow 2.2.0 breaks compatibility here
        writer = self._train_writer
        
        if self.train_batches > 0:
            with self._train_writer.as_default():
                tf.summary.scalar("mean_loss", self.mean_train_loss/self.train_batches, step = self._epoch)

        if self.test_batches > 0:
            tf.print(f"Mean losses: train:{self.mean_train_loss/self.train_batches} val:{self.mean_test_loss/self.test_batches}")
            with self._val_writer.as_default():
                tf.summary.scalar("mean_loss", self.mean_test_loss/self.test_batches, step = self._epoch)

        self._epoch += 1
        self.train_batches = 0
        self.test_batches = 0
        self.mean_train_loss = 0
        self.mean_test_loss = 0

    def on_train_batch_end(self, batch, logs=None):
        self.train_batches += 1
        if "loss" in logs:
            self.mean_train_loss += logs["loss"]

    def on_test_batch_end(self, batch, logs=None):
        self.test_batches += 1
        if "loss" in logs:
            self.mean_test_loss += logs["loss"]
  
            

In [ ]:
tboard_callback = MeanTBCallback(log_dir = "/tmp/tflogs", histogram_freq=5, profile_batch=0) #LOG_DIR

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(MODEL_SAVE_NAME + "/{epoch:02d}-{loss:.2f}-siamese", 
#                                                          monitor="loss", save_best_only=True)
# tf.debugging.experimental.enable_dump_debug_info(
#     "/tmp/tfdbg2_logdir", tensor_debug_mode="FULL_HEALTH",
#     circular_buffer_size=1000, op_regex=None, tensor_dtypes=None
# )


In [ ]:
# Epochs to train
train_for = 1600
validation_interval = 10
save_interval = 120
try:
    for i in range(last_epoch, last_epoch+train_for):
        validation = {}
        if last_epoch % validation_interval == 0:
            validation = {
                "validation_data":validDset,
                "validation_steps":2,#validLength//BATCH_SIZE, 
                "validation_batch_size":BATCH_SIZE
            }
        else:
            validation = {
                "validation_data":None,
                "validation_steps":None, 
                "validation_batch_size":None
            }
        triplet_model.fit(trainDset, 
                      initial_epoch=last_epoch,
                      epochs=last_epoch+1,
                      steps_per_epoch=2,#trainLength//BATCH_SIZE,
                      callbacks=[tboard_callback],#, checkpoint_callback]
                      **validation
                     ) # batch_size unspecified since it's generated by generator
        last_epoch += 1
        
        if last_epoch%save_interval == 0:
            if MODEL_SAVE_NAME is not None:
                siamese_base.save(MODEL_SAVE_NAME + "_" + str(last_epoch))
except KeyboardInterrupt as e:
    print("Interrupted")


In [ ]:
    
# if MODEL_SAVE_NAME is not None:
#     siamese_base.save(MODEL_SAVE_NAME + "_" + str(last_epoch))

In [ ]:
reprs = siamese_base.predict(tf.image.central_crop(imagesTable, 224/256))


In [ ]:
norm_reps = reprs / tf.expand_dims(tf.maximum(tf.math.reduce_max(reprs, axis=1), 1e-7), axis=1)

In [ ]:
selected_images = [v for a in [vv[1] for vv in validationSubset] for v in a]
fig, plots = plt.subplots(len(selected_images), 6, figsize=(110/12/2,130*len(selected_images)/72/2))
for i, selected in enumerate(selected_images):
    img = norm_reps[selected]
    dist = lambda x: tf.sqrt(tf.reduce_sum((x - img)**2))
    reprs_distance = tf.map_fn(dist, norm_reps)
    closest_idx = np.argsort(reprs_distance)
    for j, p in enumerate(closest_idx[0:6]):
        plots[i][j].imshow(imagesTable[p].numpy())
        if j == 0:
            plots[i][j].set_title(f"{p}", fontsize=7)
        else:
            plots[i][j].set_title(f"{reprs_distance[p]:2.3}", fontsize=7)
        plots[i][j].axes.get_xaxis().set_visible(False)
        plots[i][j].axes.get_yaxis().set_visible(False) 

In [ ]:
plt.hist(np.log10(np.ravel(reprs)[np.ravel(reprs) != 0.0]))

In [ ]:
validation = {
    "validation_data":validDset,
    "validation_steps":validLength//BATCH_SIZE, 
    "validation_batch_size":BATCH_SIZE
}
triplet_model.fit(trainDset, 
              epochs=1,
              steps_per_epoch=1,
              **validation
             )

In [ ]:
triplet_model.evaluate(validDset, steps=validLength//BATCH_SIZE)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $LOG_DIR

In [ ]:
[x for y in a for x in y]